### Importations

In [21]:
import orbit_modelling_library as orbit
import math as m

# Fonction $\chi^2$

In [22]:
def chi_2(observation, model, delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed):
    if len(observation.t) != len(model.t):
        print("Error : the observation and the model are not sampled at the same times")
        return
    for i in range(len(observation.t)):
        if observation.t[i] != model.t[i]:
            print("Error : the observation and the model are not sampled at the same times")
            return
    
    chi_2 = 0
    delta_position = delta_initial_position
    delta_speed = delta_initial_speed
    T = observation.t
    t0 = T[0]
    time_step = T[1] - T[0]
    
    for i in range(len(T)):
        X_obs, X_mod = observation.X[i], model.X[i]
        Y_obs, Y_mod = observation.Y[i], model.Y[i]
        Z_obs, Z_mod = observation.Z[i], model.Z[i]
        
        delta_position += time_step * delta_speed
        delta_speed += time_step * delta_accelerometer
        
        chi_2 += ((X_obs - X_mod)**2 + (Y_obs - Y_mod)**2 + (Z_obs - Z_mod)**2) / (delta_radar**2 + delta_position**2)
    return chi_2 

# Test de la fonction $\chi^{2}$

In [25]:
observation = orbit.Orbit_model("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**6, "Yukawa_1_month", type = "Yukawa")
model = orbit.Orbit_model("2050-01-01T00:00:00", "2050-02-01T00:00:00", 10**6, "all_forces_1_month", type = "Newton")
observation.load()
model.load()

delta_radar = 1
delta_accelerometer = 10**(-12)
delta_initial_position = delta_radar
delta_initial_speed = 0

print(chi_2(observation, model, delta_radar, delta_accelerometer, delta_initial_position, delta_initial_speed))

4131493.0072156894


In [26]:
m.sqrt(delta_radar + 0.5*delta_accelerometer*(86400*30)**2)

2.087877391036169

Pour rappel, on avait obtenu une distance finale entre ces 2 trajectoires de 15m, soit environ 7 $\sigma$ d'après le calcul de la cellule précédente. Comme on trouve une certitude à 2000 $\sigma$, on pense qu'il y a une erreur quelque part...